## Problem formulation


Given time series data for a small period of time (say, 2 seconds), determine whether chewing movement exists.

In [1]:
import pandas as pd 

test_csv = pd.read_csv('./data/Test_1ANoeatHeadStill.csv')
test_csv.shape

(1582, 17)

Our data has 17 columns. So formally, each data point is a vector $x \in R^{n}$. Here for this case $n = 17$. 

To utilize the data, we can use some weakly-supervised approach. Assumption: any 3 seconds in "Eat" data will definitely have chewing movement, any 3 seconds in "NoEat" data will have no chewing movement at all. So we can cut our data into lots of 3-second period to train the model. This is merely just a small trick to get more data. No data no learning no training no accuracy.

A 3-second period translates to 80 consecutive data samples in the original data. Let $m = 80$ for the sake of generality. Our task is to calculate $P(A)$ where $A \in R^{m \times n}$, meaning the probability of a 3-second period ($A$) to have chewing movement.

## Data preparation

I'm using Tensorflow here. Basically this part convert the original data to the data format that we want, do sort of normalization and stuffs.

In [2]:
paths = {
    "positive": [
        "./data/Test_1BEatHeadStill.csv",
        "./data/Test_1CChewGuava.csv",
        "./data/Test_2AEatMoving.csv",
        "./data/Test_2CEatNhan.csv",
    ],
    "negative": [
        "./data/Test_1ANoeatHeadStill.csv",
        "./data/Test_2ANoeatMoving.csv",
    ]
}


In [3]:
from dataset import Dataset
dataset = Dataset(paths, window_size=80, batch_size=32, shuffle=True).dataset
dataset

2023-10-21 12:31:32.885 | INFO     | dataset:__init__:12 - Building dataset with window_size=80, batch_size=32, shuffle=True, buffer_size=1000
2023-10-21 12:31:33.108 | INFO     | dataset:__init__:20 - Built dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 80, 17), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 80, 17), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>

## Model

In [4]:
from model import create_model
model = create_model(dataset_shape=dataset.element_spec[0].shape, verbose=5)
# TODO: wth did it run so many times?

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 71, 100)           17100     
                                                                 
 conv1d_1 (Conv1D)           (None, 62, 100)           100100    
                                                                 
 max_pooling1d (MaxPooling1  (None, 20, 100)           0         
 D)                                                              
                                                                 
 conv1d_2 (Conv1D)           (None, 11, 160)           160160    
                                                                 
 conv1d_3 (Conv1D)           (None, 2, 160)            256160    
                                                                 
 global_average_pooling1d (  (None, 160)               0         
 GlobalAveragePooling1D)                                

## Training

In [5]:
from model import fit  
fit(model, dataset, epochs=10)

2023-10-21 12:31:33.483 | INFO     | model:fit:31 - Fitting model for 10 epochs
2023-10-21 12:31:33.483 | INFO     | model:fit:31 - Fitting model for 10 epochs
2023-10-21 12:31:33.483 | INFO     | model:fit:31 - Fitting model for 10 epochs
Epoch 1/10
184/184 [==============================] - 9s 39ms/step - loss: 12708403.0000 - accuracy: 0.7994
Epoch 2/10
184/184 [==============================] - 7s 37ms/step - loss: 0.6970 - accuracy: 0.4516
Epoch 3/10
184/184 [==============================] - 7s 40ms/step - loss: 0.6968 - accuracy: 0.4693
Epoch 4/10
184/184 [==============================] - 7s 39ms/step - loss: 0.6968 - accuracy: 0.4700
Epoch 5/10
184/184 [==============================] - 7s 38ms/step - loss: 0.6967 - accuracy: 0.4560
Epoch 6/10
184/184 [==============================] - 7s 38ms/step - loss: 0.6966 - accuracy: 0.4646
Epoch 7/10
184/184 [==============================] - 7s 38ms/step - loss: 0.6966 - accuracy: 0.4506
Epoch 8/10
184/184 [==========================